In [ ]:
from dotenv import load_dotenv
from langchain_community.llms import YandexGPT
import asyncio

load_dotenv()

In [ ]:
llm = YandexGPT(
    temperature=0,
    model_name="yandexgpt"
)

In [ ]:
from langchain.chains import LLMMathChain # Цепочка, интерпретирующая промпт и запускающая интерпретатор Python для вычислений
from langchain.agents import Tool

In [ ]:
# initialize the math tool

llm_math = LLMMathChain.from_llm(llm)

math_tool = Tool(
    name='Калькулятор',
    func=llm_math.invoke,
    coroutine=llm_math.ainvoke,
    description='Полезно, когда необходимо решить вопросы с математическими выражениями.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [ ]:
await zero_shot_agent.ainvoke("Чему равен квадратный корень из 1215")

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Поиск",
    description="полезно, когда нужно произвести поиск информации в Интернете"
)

In [ ]:
tools = [search_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [ ]:
agent.invoke("""Что происходило со связью в Москве во время Парада Победы 2025 года?""")